# *Eat Safe, Love*

## Instructions
The UK Food Standards Agency evaluates various establishments across the United Kingdom, and gives them a food hygiene rating. You've been contracted by the editors of a food magazine, *Eat Safe, Love*, to evaluate some of the ratings data in order to help their journalists and food critics decide where to focus future articles.

## Part 1: Database and Jupyter Notebook Set Up

In [101]:
# Ensure pymongo is installed in the current Jupyter Notebook environment
# !pip install pymongo

In [102]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint
import json


In [103]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [104]:
# Checking the available database names
database_names = mongo.list_database_names()

# Print the list of database names
print(database_names)

['admin', 'classDB', 'config', 'fruits_db', 'local', 'met', 'petsitly_marketing', 'travel_db', 'uk_food']


In [105]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

# Perform an operation on the 'uk_food' database as an empty database will not be listed
collection = db['confirming_exists_collection']
collection.insert_one({"confirmation_exists": "true"})

# Check that 'uk_food' database exists.
database_names = mongo.list_database_names()
print(database_names)


['admin', 'classDB', 'config', 'fruits_db', 'local', 'met', 'petsitly_marketing', 'travel_db', 'uk_food']


In [106]:
# review the collections in the 'uk_food' database
collection_names = db.list_collection_names()

# Print the list of collection names
for collection_name in collection_names:
    print(collection_name)

establishments
confirming_exists_collection


In [107]:
# Add establishments.json as a new collection in the 'uk_food' database.
with open('Resources/establishments.json') as file:
    data = json.load(file)

# Insert into the collection
collection = db['establishments']
result = collection.insert_many(data)

# Print the list of collection names
for collection_name in collection_names:
    print(collection_name)

establishments
confirming_exists_collection


In [108]:
# Clean database by dropping the collection [confirming_exists_collection]
drop_collection = db['confirming_exists_collection']
drop_collection.drop()

# Print the list of collection names
collection_names = db.list_collection_names()
for collection_name in collection_names:
    print(collection_name)

establishments


In [109]:
# review a document in the establishments collection
# Query and retrieve a single document
document = collection.find_one()

# Print the document
pprint(document)

{'AddressLine1': 'East Cliff Pavilion',
 'AddressLine2': 'Wear Bay Road',
 'AddressLine3': 'Folkestone',
 'AddressLine4': 'Kent',
 'BusinessName': 'The Pavilion',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4591.765489457773,
 'FHRSID': 1043695,
 'LocalAuthorityBusinessID': 'PI/000073616',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6BL',
 'RatingDate': '2018-04-04T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': 5,
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('6491fee37e5bbe02062e5a07'),
 'geocode': {'latitude': 51.083812, 'longitude': 1.195625},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/1043695',
            'rel': 'self'}],
 'meta': {'dataSource': None,


## Part 2: Update the Database

The magazine editors have some requested modifications for the database before you can perform any queries or analysis for them. Make the following changes to the *establishments* collection:

An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following information to the database:

{
    "BusinessName":"Penang Flavours",
    "BusinessType":"Restaurant/Cafe/Canteen",
    "BusinessTypeID":"",
    "AddressLine1":"Penang Flavours",
    "AddressLine2":"146A Plumstead Rd",
    "AddressLine3":"London",
    "AddressLine4":"",
    "PostCode":"SE18 7DY",
    "Phone":"",
    "LocalAuthorityCode":"511",
    "LocalAuthorityName":"Greenwich",
    "LocalAuthorityWebSite":"http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress":"health@royalgreenwich.gov.uk",
    "scores":{
        "Hygiene":"",
        "Structural":"",
        "ConfidenceInManagement":""
    },
    "SchemeType":"FHRS",
    "geocode":{
        "longitude":"0.08384000",
        "latitude":"51.49014200"
    },
    "RightToReply":"",
    "Distance":4623.9723280747176,
    "NewRatingPending":True
}

- Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields.

- Update the new restaurant with the BusinessTypeID you found.

- The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

- Some of the number values are stored as strings, when they should be stored as numbers.

- Use update_many to convert latitude and longitude to decimal numbers.
- Use update_many to convert RatingValue to integer numbers.

In [110]:
# assign the collection to a variable
establishments = db['establishments']

In [111]:
# Add the new restaurant to the database
# Create a dictionary for the new restaurant data
document_halal_restaurant = {
    "BusinessName": "Penang Flavours",
    "BusinessType": "Restaurant/Cafe/Canteen",
    "BusinessTypeID": "",
    "AddressLine1": "Penang Flavours",
    "AddressLine2": "146A Plumstead Rd",
    "AddressLine3": "London",
    "AddressLine4": "",
    "PostCode": "SE18 7DY",
    "Phone": "",
    "LocalAuthorityCode": "511",
    "LocalAuthorityName": "Greenwich",
    "LocalAuthorityWebSite": "http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress": "health@royalgreenwich.gov.uk",
    "scores": {
        "Hygiene": "",
        "Structural": "",
        "ConfidenceInManagement": ""
    },
    "SchemeType": "FHRS",
    "geocode": {
        "longitude": "0.08384000",
        "latitude": "51.49014200"
    },
    "RightToReply": "",
    "Distance": 4623.9723280747176,
    "NewRatingPending": True
}


In [112]:
# Insert the new restaurant into the collection
halal_restaurant = collection.insert_one(document_halal_restaurant)

In [113]:
# Check that the new restaurant was inserted by checking the document's ID
print("Inserted document ID:", halal_restaurant.inserted_id)

Inserted document ID: 64bad4894a4c7e55f9c3850f


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [114]:
# # Retrieve all documents from the collection
all_documents = establishments.find()

In [115]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
# Keep track of previously seen BusinessType and BusinessTypeID
previous_business_type = None
previous_business_id = None

# Define the filter
filter = {"BusinessType": "Restaurant/Cafe/Canteen"}

# Define the projection to include only the desired fields
projection = {"BusinessTypeID": 1, "BusinessType": 1, "_id": 0}

# Keep track of unique combinations of BusinessTypeID and BusinessType
unique_combinations = set()

# Execute the query and retrieve the matching documents
result = establishments.find(filter, projection)

# Iterate over the result and print the BusinessTypeID and BusinessType fields when a new unique combination appears
for document in result:
    business_type = document.get("BusinessType")
    business_id = document.get("BusinessTypeID")
    
    combination = (business_type, business_id)
    
    if combination not in unique_combinations:
        print("BusinessTypeID:", business_id)
        print("BusinessType:", business_type)
        print()
        
        unique_combinations.add(combination)

BusinessTypeID: 1
BusinessType: Restaurant/Cafe/Canteen

BusinessTypeID: 
BusinessType: Restaurant/Cafe/Canteen



3. Update the new restaurant with the `BusinessTypeID` you found.

In [116]:
# Update the new restaurant with the correct BusinessTypeID
# Define the filter to match the specific document
filter = {"BusinessName": "Penang Flavours"}

# Define the update operation to set the new value for BusinessTypeID
update = {"$set": {"BusinessTypeID": 1}}

# Update the document that matches the filter with the new value
result = establishments.update_one(filter, update)

In [117]:
# Confirm that the new restaurant was updated
# Define the filter to match the specific document
filter = {"BusinessName": "Penang Flavours"}

# Query the collection to retrieve the updated document
updated_document = establishments.find_one(filter)

# Print the updated document
pprint(updated_document)

{'AddressLine1': 'Penang Flavours',
 'AddressLine2': '146A Plumstead Rd',
 'AddressLine3': 'London',
 'AddressLine4': '',
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'Distance': 4623.972328074718,
 'LocalAuthorityCode': '511',
 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
 'LocalAuthorityName': 'Greenwich',
 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
 'NewRatingPending': True,
 'Phone': '',
 'PostCode': 'SE18 7DY',
 'RatingValue': None,
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('6492157f34fda249327574f1'),
 'geocode': {'latitude': 51.490142, 'longitude': 0.08384},
 'scores': {'ConfidenceInManagement': '', 'Hygiene': '', 'Structural': ''}}


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [118]:
# Find how many documents have LocalAuthorityName as "Dover"
# Define the filter to match the documents with LocalAuthorityName as "Dover"
filter = {"LocalAuthorityName": "Dover"}

# Find the documents matching the filter and get the count
count = establishments.count_documents(filter)

# Print the count of documents
print("Number of documents with LocalAuthorityName as 'Dover':", count)

Number of documents with LocalAuthorityName as 'Dover': 994


In [119]:
# Delete all documents where LocalAuthorityName is "Dover"
# Define the filter to match the documents with LocalAuthorityName as "Dover"
filter = {"LocalAuthorityName": "Dover"}

# Delete the documents matching the filter
result = establishments.delete_many(filter)

# Check the result of the delete operation
print("Number of documents deleted:", result.deleted_count)

Number of documents deleted: 994


In [120]:
# Define the filter to match documents with LocalAuthorityName containing "Dover"
filter = {"LocalAuthorityName": {"$regex": "Dover"}}

# Execute the query and retrieve the matching documents
result = establishments.count_documents(filter)

# Check if any documents were found
if result > 0:
    print("There are remaining documents with LocalAuthorityName including 'Dover'")
else:
    print("There are no remaining documents with LocalAuthorityName including 'Dover'")


There are no remaining documents with LocalAuthorityName including 'Dover'


In [121]:
# Check that other documents remain with 'find_one'
remaining_documents = establishments.find_one()
remaining_documents

{'_id': ObjectId('6491fee37e5bbe02062e5a07'),
 'FHRSID': 1043695,
 'ChangesByServerID': 0,
 'LocalAuthorityBusinessID': 'PI/000073616',
 'BusinessName': 'The Pavilion',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'AddressLine1': 'East Cliff Pavilion',
 'AddressLine2': 'Wear Bay Road',
 'AddressLine3': 'Folkestone',
 'AddressLine4': 'Kent',
 'PostCode': 'CT19 6BL',
 'Phone': '',
 'RatingValue': 5,
 'RatingKey': 'fhrs_5_en-gb',
 'RatingDate': '2018-04-04T00:00:00',
 'LocalAuthorityCode': '188',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'scores': {'Hygiene': 5, 'Structural': 5, 'ConfidenceInManagement': 5},
 'SchemeType': 'FHRS',
 'geocode': {'longitude': 1.195625, 'latitude': 51.083812},
 'RightToReply': '',
 'Distance': 4591.765489457773,
 'NewRatingPending': False,
 'meta': {'dataSource': None,
  'extractDate': '0001-01-01T00:00

5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [122]:
# Change the data type from String to Decimal for longitude and latitude
establishments.update_many(
    {},
    [
        {
            '$set': {
                'geocode.longitude': {
                    '$toDouble': '$geocode.longitude'
                },
                'geocode.latitude': {
                    '$toDouble': '$geocode.latitude'
                }
            }
        }
    ]
)

Use `update_many` to convert `RatingValue` to integer numbers.

In [123]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

In [124]:
# Change the data type from String to Integer for RatingValue
establishments.update_many(
    {},
    [
        {
            '$set': {
                'RatingValue': {
                    '$toInt': '$RatingValue'
                }
            }
        }
    ]
)

In [125]:
# Check that the coordinates and rating value are now numbers
collection = db.establishments

# Query the documents
documents = collection.find()

# Initialize flag variables to track if any message was printed
coordinate_message_printed = False
rating_message_printed = False

# Iterate over the documents and check the data types
for doc in documents:
    longitude = doc['geocode']['longitude']
    latitude = doc['geocode']['latitude']
    rating_value = doc['RatingValue']
    
    # Convert None to 0
    if longitude is None:
        longitude = 0
    if latitude is None:
        latitude = 0
    if rating_value is None:
        rating_value = 0
    
    # Check the data types
    if isinstance(longitude, float) and isinstance(latitude, float):
        # Coordinates are numbers
        coordinate_message_printed = True

    if isinstance(rating_value, int):
        # RatingValue is an integer
        rating_message_printed = True
    
    if not coordinate_message_printed and not rating_message_printed:
        print(f"{longitude} & {rating_value}: Not all coordinates or RatingValue values are numbers.")

# Check if all values are integers and no message was printed
if coordinate_message_printed and rating_message_printed:
    print('Confirmed: Coordinates and RatingValue are numbers and integers.')

Confirmed: Coordinates and RatingValue are numbers and integers.
